In [ ]:
pip install git+https://github.com/boudinfl/pke.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-04f88131
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-04f88131
  Resolved https://github.com/boudinfl/pke.git to commit f2d4f5d2252c64d23defccd32fdac8809cfd7ce0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.4 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160676 sha256=90f210ecb771847f138999b94ce37c5539041051b72743d887da83f667d4356b
  Stored in directory: /tmp/pip-ephem-wheel-cache-a3ojslid/wheels/d5/46/97/85535b5b449f70b6a3c8d1138ce8587345876891e25bfe7954
Successfully built pke


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install flashtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9308 sha256=5851f6860a692c047676a00bd363aa947d9840935985637ad895a3a43ae3fcc7
  Stored in directory: /root/.cache/pip/wheels/65/3c/c7/44672c5062c16d05760b1eaddbf611d2f6a4b715c6d6777418
Successfully built flashtext


In [ ]:
import nltk
nltk.download("stopwords")
import string
import pke
from nltk.corpus import stopwords
import collections
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor
import pandas as pd


def Keyword_Extraction(text):
  #TF-IDF
  extractor = pke.unsupervised.TfIdf()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)
  extractor.candidate_selection(n=3)
  extractor.candidate_weighting()

  keyphrases_TFIDF = extractor.get_n_best(n=10)

  ###############################################################

  #KPMiner
  extractor = pke.unsupervised.KPMiner()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)
  lasf = 5
  cutoff = 200
  extractor.candidate_selection(lasf=lasf, cutoff=cutoff)
  alpha = 2.3
  sigma = 3.0
  extractor.candidate_weighting(alpha=alpha, sigma=sigma)
  keyphrases_KPMiner = extractor.get_n_best(n=10)

  ###########################################################

  #YAKE
  extractor = pke.unsupervised.YAKE()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)
  stoplist = stopwords.words('english')
  extractor.candidate_selection(n=3)
  window = 2
  use_stems = False
  extractor.candidate_weighting(window=window,
                                use_stems=use_stems)
  threshold = 0.8
  keyphrases_Yake = extractor.get_n_best(n=10, threshold=threshold)

  ################################################################

  #TextRank
  pos = {'NOUN', 'PROPN', 'ADJ'}
  extractor = pke.unsupervised.TextRank()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)
  extractor.candidate_weighting(window=2,
                                pos=pos,
                                top_percent=0.33)
  keyphrases_TextRank = extractor.get_n_best(n=10)

  pos = {'NOUN', 'PROPN', 'ADJ'}

  ##############################################################

  # SingleRank extractor.
  extractor = pke.unsupervised.SingleRank()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)

  extractor.candidate_selection(pos=pos)
  extractor.candidate_weighting(window=10,
                                pos=pos)
  keyphrases_SingleRank = extractor.get_n_best(n=10)

  ##############################################################

  #TopicRank
  extractor = pke.unsupervised.TopicRank()
  extractor.load_document(input=text)

  pos = {'NOUN', 'PROPN', 'ADJ'}
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.candidate_selection(pos=pos)
  extractor.candidate_weighting(threshold=0.74, method='average')
  keyphrases_TopicRank = extractor.get_n_best(n=10)

  ######################################################################

  pos = {'NOUN', 'PROPN', 'ADJ'}
  grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

  #TopicalPageRank extractor.
  extractor = pke.unsupervised.TopicalPageRank()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)
  extractor.candidate_selection(grammar=grammar)

  extractor.candidate_weighting(window=10,pos=pos)
  keyphrases_TopicalPageRank = extractor.get_n_best(n=10)

  ############################################################################

  pos = {'NOUN', 'PROPN', 'ADJ'}
  grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"

  #PositionRank extractor.
  extractor = pke.unsupervised.PositionRank()
  extractor.load_document(input=text,
                          language='en',
                          normalization=None)
  extractor.candidate_selection(grammar=grammar,
                                maximum_word_number=3)
  extractor.candidate_weighting(window=10,
                                pos=pos)
  keyphrases_PositionRank = extractor.get_n_best(n=10)

  #################################################################

  # MultipartiteRank extractor.
  extractor = pke.unsupervised.MultipartiteRank()
  extractor.load_document(input=text)
  pos = {'NOUN', 'PROPN', 'ADJ'}
  stoplist = list(string.punctuation)
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  extractor.candidate_selection(pos=pos)
  extractor.candidate_weighting(alpha=1.1,
                                threshold=0.74,
                                method='average')
  keyphrases_MultipartiteRank = extractor.get_n_best(n=10)

  ####################################################################

  stoplist = stopwords.words('english')

  #Kea extractor.
  extractor = pke.supervised.Kea()

  extractor.load_document(input=text,
                          language='en',
                          normalization=None)

  extractor.candidate_selection()
  extractor.candidate_weighting()
  keyphrases_Kea = extractor.get_n_best(n=10)

  #######################################################################

  #WINGNUS extractor.
  extractor = pke.supervised.WINGNUS()
  extractor.load_document(input=text)
  extractor.candidate_selection()
  extractor.candidate_weighting()
  keyphrases_WINGNUS = extractor.get_n_best(n=10)

  ########################################################################

  Keywords= keyphrases_TFIDF + keyphrases_KPMiner + keyphrases_Yake + keyphrases_TextRank +keyphrases_SingleRank + keyphrases_TopicRank + keyphrases_TopicalPageRank +keyphrases_PositionRank + keyphrases_MultipartiteRank + keyphrases_Kea + keyphrases_WINGNUS
  Keywords = [k for k,v in Keywords]

  ########################################################################
  duplicated_Keywords=[item for item, count in collections.Counter(Keywords).items() if count > 1]
  return duplicated_Keywords

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]

    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(Keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in Keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
        df = pd.DataFrame({'values':values})
        df.drop_duplicates(inplace = True)
        keyword_sentences[key] = df['values'].tolist()

    return keyword_sentences

def keyword_sentenceMapping(text):
  sentences = tokenize_sentences(text)
  duplicated_Keywords = Keyword_Extraction(text)
  keyword_sentence_mapping_after = get_sentences_for_keyword(duplicated_Keywords, sentences)
  keyword_sentence_mapping={key: value for key, value in keyword_sentence_mapping_after.items() if value}
  return keyword_sentence_mapping,duplicated_Keywords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# transcript = "Electromagnetism, or the electromagnetic force, is one of the four fundamental forces of nature. These electric forces bring and hold atoms together. If those electrically charged particles start to move, the field will become a flowing electric current and form a magnetic field around it. On the other are short, high-frequency wavelengths, ultraviolet, ex, and gamma radiation. Unlike low-frequency waves, these waves can pass through the human body, making them useful for medical applications."
# transcript = "Electromagnetism, or the electromagnetic force, is one of the four fundamental forces of nature. Electromagnetism is a branch of physics that studies the interactions between electric and magnetic fields. All matter has an electric charge, which can be positive, negative, or zero. And when atoms gain a positive or negative charge through the transfer of electrons, a measurable electric field will form. The intensity of waves of electromagnetic energy or radiation is determined by waves of electromagnetic energy or radiation's frequency. Near the middle of the electromagnetic spectrum is visible light, such as light emitted by stars, fireflies, and computer screens. On either side are invisible electromagnetic waves. On one end are long, low-frequency radio waves that broadcast television and radio signals, micro waves that carry telephone signals and cook food, and infrared waves emitted by fires. Unlike low-frequency waves, short, high-frequency wavelengths, ultraviolet, ex, and gamma radiation can pass through the human body, making short, high-frequency wavelengths, ultraviolet, ex, and gamma radiation useful for medical applications. While electromagnetism has daily applications, electromagnetism's also the engine that drives the world as we know it. Called a geodynamo, this process causes the world as we know it's poles to attain positive and negative charges, turning the world as we know it into a giant electromagnet. a protective layer around the planet shields us from the most harmful radiation in space, leaving us to enjoy a world held together by Electromagnetism, or the electromagnetic force,."

transcript = "Our solar system is one of over 500 known solar systems in the entire Milky Way galaxy. Our solar system is located in the entire Milky Way galaxy's Orion Star Cluster. Only 15% of stars in the entire Milky Way galaxy host planetary systems, and one of Only 15% of stars in the galaxy is our own sun. Revolving around our own sun are eight planets. eight planets are divided into two categories based on eight planets's composition, terrestrial and jovian. Real planets, including Mercury, Venus, Earth and Mars's surfaces are solid, Real planets, including Mercury, Venus, Earth and Mars don't have ring systems, Real planets, including Mercury, Venus, Earth and Mars have very few or no moons, and Real planets, including Mercury, Venus, Earth and Mars are relatively small. The smallest and closest to our own sun is Mercury, which has the shortest orbit in Our solar system at about three Earth months. Venus is the hottest planet, with temperatures of up to 867 degrees Fahrenheit due to an atmosphere of carbon dioxide and extensive lava flows. The water systems on Earth help create the only known environment in the universe capable of sustaining life. The last of eight planets, Mars, might have also supported life about 3.7 billion years ago when Mars had a watery surface and moist atmosphere. Beyond the four terrestrial planets of the inner solar system, lie the jovian planets of the outer solar system. the jovian planets of the outer solar system include gas giants Jupiter and Saturn and ice giants Uranus and Neptune. gas giants Jupiter and Saturn are predominantly made of helium and hydrogen, and the jovian planets of the outer solar system also contain rock, ice and a liquid mixture of water, methane and ammonia. the jovian planets of the outer solar system have multiple moons, sport ring systems, have no solid surface and are immense. Saturn's signature rings are wide enough to fit between Earth and the Moon, but are fairly a kilometer thick. Past Saturn are the ice giants Uranus and Neptune. The slightly bigger of ice giants Uranus and Neptune Uranus is famous for rotating on Uranus's side. Next Uranus is Neptune, the outermost planet in Our solar system and also one of the coldest. Orbiting the four terrestrial planets of the inner solar system is the asteroid belt, a flat disk of rocky objects full of remnants from Our solar system's formation, from microscopic dust particles to the largest known object, the dwarf planet Ceres. Beyond the icy Kuiper belt is the Ork cloud, a vast spherical collection of icy debris. the Ork cloud, a vast spherical collection of icy debris is considered the edge of Our solar system since the edge of the solar system is where the gravitational and physical influences of our own sun end. Our solar system's particular configuration of planets and other celestial objects, all revolving around a life-giving star, make Our solar system a special place to call home."

In [ ]:
# transcript = "a green color component called chloroplasts is responsible for the green color of plants and leaves. Leaves on the plant have pores, very similar to the pores on the skin of our bodies. This is because energy from the sun is absorbed by the water layers and only some amount of energy actually reaches the plant. Do kids know how plants make plants's own food? No? Well, this video elaborates the process of photosynthesis by which plants make plants's own food. chemical energy is used by plants for growth and nourishment. Once the carbon dioxide and the water reach a green color component called chloroplasts in the presence of sunlight, the process of photosynthesis starts to take place. The carbon dioxide present in the air, which is responsible for Photosynthesis, enters the plant through pores, very similar to the pores on the skin of our bodies. This system transports vital nutrients to the plant. pores, very similar to the pores on the skin of our bodies are called stomata. The following reaction takes place in the leaves of the plant during the process of photosynthesis. Photosynthesis is the conversion of light energy into chemical energy by plants. Leaves have important cells called mesofil cells. Plants have tubes called xylum, located in the stems through which the water from the ground is sucked into the leaves. This system is spread throughout the different parts of the plant, including the stem, branches, and all the way up to the leaves. Xylums in the plant are like blood vessels in the human body that act as an important means of transportation of water and nutrients. Glucose is used by the plants for the plants's growth. The oxygen that is released is used by human beings to breathe during human beings's respiration process ever wondered why the process of photosynthesis is called photosynthesis? Well the word photosynthesis is a combination of two Greek words photo and synthesis. important cells called mesofil cells contain a green color component called chloroplasts. This system works similar to humans sucking liquids through a straw. Photosynthesis is the process used by plants to make plants's own food or in more complex terms."
transcript = "Do kids know how plants make plants's own food? Photosynthesis is the process used by plants to make plants's own food or in more complex terms. chemical energy is used by plants for growth and nourishment. Photo means light and synthesis means putting together in Greek. pores, very similar to the pores on the skin of our bodies are responsible for the exchange of gases. Carbohydrates, which are a form of glucose, are synthesized from carbon dioxide and water. Glucose is used by the plants for the plants's growth. So the essential factors for photosynthesis to take place include sunlight, water and carbon dioxide. some plants which don't need the process of photosynthesis to grow include mushroom, venous fly traps etc."

In [ ]:
# transcript = "Vitamins're the body's builders, defenders, and maintenance workers, helping the body's to build muscle and bone, make use of nutrients, capture and use energy, and heal wounds. On the other hand, too much of any vitamin can cause toxicity in the body, so there goes the myth that loading yourself with supplements is a great idea. The water-solubles are vitamin C and B-complex vitamins that are made up of eight different types that each do something unique. Vitamin D gathers calcium and phosphorus so we can make bones, and vitamin E works as an antioxidant, getting rid of elements in the body that can damage cells. Because fat-soluble vitamins can't make use of the blood's watery nature, fat-soluble vitamins need something else to move fat-soluble vitamins around. Vitamins come in two types, lipid-soluble and water-soluble, and the difference between two types, lipid-soluble and water-soluble determines how the body transports and stores vitamins and gets rid of the excess. So how does the body get vitamins from out there into here? That's dependent on the form vitamins take. Because blood plasma is water-based, water-soluble vitamins C and B have water-soluble vitamins C and B's transport cut out for water-soluble vitamins C and B, and can move around freely within the body. Vitamins are organic compounds we need to ingest in small amounts to keep functioning. The body treats these parts like a pantry, storing fat-soluble vitamins there and rationing fat-soluble vitamins out when needed, meaning we shouldn't overload on this type of vitamin because The body is generally well stocked. And just like letters build words, letters're the building blocks that keep the body running. lipid-soluble vitamins, dissolved in fat and found in foods like dairy, butter, and oils make it through the stomach and the intestine, where an acidic substance called bile flows in from the liver, breaking up the fat and preparing the fat for absorption through the intestinal wall. So this difference between water or fat-soluble vitamins determines how fat-soluble vitamins get into the blood, but also how fat-soluble vitamins're stored or rejected from the body. The water-solubles are dissolved in the watery parts of fruits, vegetables, and grains, meaning The water-solubles's passage through the body is relatively straightforward."
transcript = "A, C, E, D, B, K. No, this isn't some random out-of-order alphabet. That's dependent on the form these compounds take. Vitamins come in two types, lipid-soluble and water-soluble, and the difference between them determines how the body transports and stores vitamins and gets rid of the excess. Once inside the system, these foods are digested, and the vitamins within them are taken up directly by the bloodstream. Because fat-soluble vitamins can't make use of the blood's watery nature, they need something else to move them around. Vitamin A helps make white blood cells, key in the body's defense, helps shape bones, and improves vision by keeping the cells of the eye in check. Without this vitamin variety, humans face deficiencies that cause a range of problems like fatigue, nerve damage, heart disorders, or diseases like rickets and scurvy."

In [ ]:
transcript = "Rutherford's atomic model was quite commendable when it came to explaining the structure of atoms. We know that any charged object which revolves in a circular motion gains acceleration gradually. Now if the electron continuously keeps radiating energy, then ultimately all the energy of the electron will get over and all the energy of the electron will fall into the nucleus. slight modifications were made by the next legendary scientist in our list called Niels Bohr. Now these points definitely tell us why an atom is so stable. Let's understand with an example. Now here, the sun is stationary at the centre while the planets revolve around the sun. Yes, K L M N and so on. With all these theories and points known, do we now know the structure of a typical atom completely? Is that how an atom is structured? Astonishingly, one more subatomic particle had no charge. In the center lies the nucleus having positively charged protons and neutral neutrons, while the negatively charged electrons revolve in fixed orbitals around The nucleus."
# transcript = "Rutherford's atomic model was quite commendable when it came to explaining the structure of atoms. However, Rutherford's atomic model was not very appropriate theoretically. Now if the electron continuously keeps radiating energy, then ultimately all the energy of the electron will get over and all the energy of the electron will fall into the nucleus. That means the hypothesis put forth by Rutherford's was also incorrect. Not really, the hypothesis put forth by Rutherford just needed slight modifications. slight modifications were made by the next legendary scientist in our list called Niels Bohr. the next legendary scientist in our list called Niels Bohr made a few additional explanations to describe the atomic structure. The postulates put forward by the next legendary scientist in our list called Niels Bohr were as follows. Firstly, only certain special orbits called discrete orbits of electrons are allowed inside the atom. Secondly, while revolving in only certain special orbits called discrete orbits of electrons, the electrons do not radiate energy. Now these points definitely tell us why an atom is so stable. But what exactly are these paths or orbits in which the electrons revolve around the nucleus? Let's understand with an example. Do you know how our solar system is? Yes, our solar system appears somewhat like this. Now here, the sun is stationary at the centre while the planets revolve around the sun. But have you noticed that the planets always revolve in fixed paths? We never find any planet jumping to a different path all of a sudden, right? the planets always encircle the sun in defined paths. The nucleus acts like the sun and the electrons are like planets which revolve in fixed defined orbitals. the next legendary scientist in our list called Niels Bohr suggested that the electrons revolving in fixed defined orbitals do not radiate energy. Now this is justified when we use the name energy levels because the name energy levels indicates that each shell has got a defined energy level. That means when the electrons revolve in fixed defined orbitals, the electrons do not liberate any form of energy. And what if we want to number fixed defined orbitals? In that case, we use the letter N in lower case and write fixed defined orbitals as the letter N equals 1, the letter N equals 2, N equals 3 and so on, beginning from the 1 next to The nucleus. So we can name fixed defined orbitals alphabetically or we can even number fixed defined orbitals. With all these theories and points known, do we now know the structure of a typical atom completely? The nucleus contains positive protons and the electrons revolving around in fixed orbitals. a famous English physicist Sir James Chadwick found that one more subatomic particle had a mass almost equivalent to that of the proton. And what about one more subatomic particle's charge? Astonishingly, one more subatomic particle had no charge. one more subatomic particle was later named as neutron, denoted by the letter N. Does now we have the complete design of an atom? In the center lies the nucleus having positively charged protons and neutral neutrons, while the negatively charged electrons revolve in fixed orbitals around The nucleus. But how exactly are the electrons distributed in fixed orbitals around the nucleus? Is there a way to find out the maximum number of electrons that one orbital can contain? Or is it that the electrons are randomly scattered in fixed orbitals around the nucleus? Let's find out the answers to these questions in our upcoming video."

In [ ]:
mapping,keywords = keyword_sentenceMapping(transcript)
print(mapping)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.20.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'charged': ['In the center lies the nucleus having positively charged protons and neutral neutrons, while the negatively charged electrons revolve in fixed orbitals around The nucleus.', 'We know that any charged object which revolves in a circular motion gains acceleration gradually.'], 'energy': ['Now if the electron continuously keeps radiating energy, then ultimately all the energy of the electron will get over and all the energy of the electron will fall into the nucleus.'], 'nucleus': ['Now if the electron continuously keeps radiating energy, then ultimately all the energy of the electron will get over and all the energy of the electron will fall into the nucleus.', 'In the center lies the nucleus having positively charged protons and neutral neutrons, while the negatively charged electrons revolve in fixed orbitals around The nucleus.'], 'structure': ['With all these theories and points known, do we now know the structure of a typical atom completely?', "Rutherford's atomic mod

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.20.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
print(keywords)

['charged', 'energy', 'nucleus', 'structure', 'points', 'revolve', 'sun', 'electron', 'atom', 'rutherford', 'atomic model', 'niels bohr', 'circular motion gains', 'typical atom', 'slight modifications', 'k l m n', 'circular motion gains acceleration', 'next legendary scientist', 'neutral neutrons', 'atoms']
